In [1]:
import pandas as pd
import spacy
from spacy.training.example import Example

<h1> HOMEWORK </h1>

### PART1

# NNS


# Utilisation de fonction is_plural

In [2]:

# Initialize a blank English model
nlp = spacy.blank("en")
def is_plural_noun(token):
    # Load the spaCy model
    if token.pos_ in [ "PROPN", "VERB", "ADJ", "ADV" , "ADP", "AUX", "CONJ", "DET", "INTJ", "NUM", "PART", "PRON", "SCONJ", "SYM", "X" ]:
        return False
    # Check if token ends with 's' and is not a single 's'
    if token.text.endswith('s') and len(token.text) > 1:
        # Check if the token is not a known exception (e.g., "is")
        if token.text.lower() not in ["is", "was", "has", "does" , "always"] :
            # Check if the token is not a possessive (e.g., "John's")
            if not (token.text.endswith("'s") or token.text.endswith("s'")):
                return True
    return False

# Define a custom pipeline component for the POS tagger
@nlp.component("custom_pos_tagger")
def custom_pos_tagger(doc):
    for token in doc:
        if is_plural_noun(token):
            token.tag_ = "NNS"  # overwrite the token tag
    return doc

# Add the custom component to the pipeline
nlp.add_pipe("custom_pos_tagger") # add last to the pipeline



# Prepare a training dataset with annotated POS tags
training_data = [
    ("Cats are interesting animals.", {"tags": ["NNS", "VBP", "JJ", "NNS", "."]}),
    ("Dogs and cats are friends.", {"tags": ["NNS", "CC", "NNS", "VBP", "NNS", "."]}),
    ("Cats are from Venus.", {"tags": ["NNS", "VBP", "IN", "NNP", "."]}),
    ("Dogs are from Mars.", {"tags": ["NNS", "VBP", "IN", "NNP", "."]}),
    ("I have three dogs.", {"tags": ["PRP", "VBP", "DT", "NNS", "."]}),
    ("I have five cars.", {"tags": ["PRP", "VBP", "DT", "NNS", "."]}),
    ("I have ten cats and a dog.", {"tags": ["PRP", "VBP", "DT", "NNS", "CC", "DT", "NN", "."]}),
    ("My cat's toys are green.", {"tags": ["PRP$", "NN", "POS", "NNS", "VBZ", "JJ", "."]}),
    ("I thought dogs' toys were green.", {"tags": ["PRP", "VBD", "NNS", "POS", "NNS", "VBD", "JJ", "."]}),
]
# Train the model
optimizer = nlp.begin_training()
losses = {}
for text, annotations in training_data:
    example = Example.from_dict(nlp.make_doc(text), annotations)
    nlp.update([example] ,  sgd=optimizer,drop=0.35, losses=losses)

# Test the custom POS tagger
doc = nlp("The city's parks are always bustling with people and their dogs. Children run and play, while adults enjoy picnics under the trees. The flowers in the gardens bloom in many colors, attracting bees and butterflies. It's a peaceful and lively place for everyone to relax and unwind")

for token in doc:
    if token.tag_ == "NNS":
        print(token.text, ":", token.tag_)


parks : NNS
dogs : NNS
adults : NNS
picnics : NNS
trees : NNS
flowers : NNS
gardens : NNS
colors : NNS
bees : NNS
butterflies : NNS


# 2 eme methode :

<h3>  Utilisation des regex  </h3>

In [3]:
import re
from spacy.training.example import Example

# Initialize a blank English model
nlp = spacy.blank("en")

# Define a custom pipeline component for the POS tagger
@nlp.component("custom_pos_tagger")
def custom_pos_tagger(doc):
    for token in doc:
        # Check if the token matches the plural noun pattern
        if re.match(r'\b\w+s\b', token.text) and token.text.lower() not in ["is", "was", "has", "does" , "always" , "various" ]:
            token.tag_ = "NNS"  # overwrite the token tag
    return doc

# Add the custom component to the pipeline
nlp.add_pipe("custom_pos_tagger")

# Prepare a training dataset with annotated POS tags
training_data = [
    ("Cats are interesting animals.", {"tags": ["NNS", "VBP", "JJ", "NNS", "."]}),
    ("Dogs and cats are friends.", {"tags": ["NNS", "CC", "NNS", "VBP", "NNS", "."]}),
    ("Cats are from Venus.", {"tags": ["NNS", "VBP", "IN", "NNP", "."]}),
    ("Dogs are from Mars.", {"tags": ["NNS", "VBP", "IN", "NNP", "."]}),
    ("I have three dogs.", {"tags": ["PRP", "VBP", "DT", "NNS", "."]}),
    ("I have five cars.", {"tags": ["PRP", "VBP", "DT", "NNS", "."]}),
    ("I have a cat and a dog.", {"tags": ["PRP", "VBP", "DT", "NN", "CC", "DT", "NN", "."]}),
    ("My cat's toy is green.", {"tags": ["PRP$", "NN", "POS", "NN", "VBZ", "JJ", "."]}),
    ("I thought dogs' toys were green.", {"tags": ["PRP", "VBD", "NNS", "POS", "NNS", "VBD", "JJ", "."]}),
]

# Train the model
optimizer = nlp.begin_training()
losses = {}
for text, annotations in training_data:
    example = Example.from_dict(nlp.make_doc(text), annotations)
    nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)

# Test the custom POS tagger
doc = nlp("The bustling market was filled with vendors selling a colorful array of fruits, vegetables, and handmade crafts")
for token in doc:
    if token.tag_ == "NNS":
        print(token.text, ":", token.tag_)


vendors : NNS
fruits : NNS
vegetables : NNS
crafts : NNS


# Methode : utilisation de corpus treebank for the train data

In [4]:
from nltk.corpus import brown
from nltk.corpus import treebank
from nltk.corpus import conll2000
import nltk
nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')

nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\rachi\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\rachi\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\rachi\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\rachi\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [13]:
# load POS tagged corpora from NLTK
treebank_corpus = treebank.tagged_sents(tagset='universal')
brown_corpus = brown.tagged_sents(tagset='universal')
conll_corpus = conll2000.tagged_sents(tagset='universal')
tagged_sentences = treebank_corpus + brown_corpus + conll_corpus
import nltk

# print first two sentences of the tagged_sentences corpus
for sent in tagged_sentences[:10]:
    print(sent)


[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')]
[('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')]
[('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('con

In [6]:
X = [] # store input sequence
Y = [] # store output sequence

for sentence in tagged_sentences:
    X_sentence = []
    Y_sentence = []
    for entity in sentence:
        X_sentence.append(entity[0])  # entity[0] contains the word
        Y_sentence.append(entity[1])  # entity[1] contains corresponding tag

    X.append(X_sentence)
    Y.append(Y_sentence)

# Print the first training sample
print('Input sequence:', str(X[0]))
print('Output sequence:', str(Y[0]))




Input sequence: ['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.']
Output sequence: ['NOUN', 'NOUN', '.', 'NUM', 'NOUN', 'ADJ', '.', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'NOUN', 'NUM', '.']


In [7]:
def adjust_tags(tags, sentence):
    # Split the sentence into words
    words = sentence.split()

    # Check if the number of words and tags differ by one
    if len(words) == len(tags):
        return tags
    elif len(words) == len(tags) - 1:
        # If there is one less tag, duplicate the last tag to match the number of words
        adjusted_tags = tags + [tags[-1]]
        return adjusted_tags
    else:
        raise ValueError("Number of words and tags do not match")

In [8]:
# Combine elements of X and Y into a single list
training_data = []
for i in range(len(X)):
    sentence = ' '.join(X[i])
    tags = Y[i]
    if len(sentence.split()) == len(tags):
        example = (sentence, {"tags": tags})
        training_data.append(example)
    else:
        tags = adjust_tags(tags, sentence)
        example = (sentence, {"tags": tags})
        training_data.append(example)

# Train the model
optimizer = nlp.begin_training()
losses = {}
for text, annotations in training_data:
    doc = nlp.make_doc(text)
    if len(doc) == len(annotations["tags"]):
        example = Example.from_dict(doc, annotations)
        nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)


# Test the trained model
doc = nlp("The city's parks are always bustling with people and their dogs. Children run and play, while adults enjoy picnics under the trees. The flowers in the gardens bloom in many colors, attracting bees and butterflies. It's a peaceful and lively place for everyone to relax and unwind")
for token in doc:
    if token.tag_ == "NNS":
        print(token.text, ":", token.tag_)


parks : NNS
dogs : NNS
adults : NNS
picnics : NNS
trees : NNS
flowers : NNS
gardens : NNS
colors : NNS
bees : NNS
butterflies : NNS


<h2> PART2  </h2>
Train a ner tagger to recognize a new entity label called “COLOR”


In [9]:
from spacy.training.example import offsets_to_biluo_tags
import random
# to ignore warnings
import warnings
warnings.filterwarnings("ignore")
# Create a spaCy model with the "ner" component
nlp = spacy.blank("en")
ner = nlp.add_pipe("ner")

# Add the new entity label "COLOR" to the NER model
ner.add_label("COLOR")

TRAIN_DATA = [
    ("The sky is blue.", {"entities": [(11, 15, "COLOR")]}),
    ("The ocean is deep blue.", {"entities": [(20, 24, "COLOR")]}),
    ("His favorite color is purple.", {"entities": [(24, 30, "COLOR")]}),
    ("The ripe apple is red.", {"entities": [(17, 20, "COLOR")]}),
    ("The card is red.", {"entities": [(11, 15, "COLOR")]}),

    ("The leaves turned golden.", {"entities": [(18, 24, "COLOR")]}),
    ("The sky at sunset is orange.", {"entities": [(20, 26, "COLOR")]}),
    ("The flag is red, white, and blue.", {"entities": [(11, 14, "COLOR"), (17, 22, "COLOR"), (29, 33, "COLOR")]}),
    ("The walls are green.", {"entities": [(14, 19, "COLOR")]}),
    ("The flowers are pink.", {"entities": [(17, 21, "COLOR")]}),
    ("The book has a brown cover.", {"entities": [(17, 22, "COLOR")]}),
    ("The car is silver.", {"entities": [(12, 18, "COLOR")]}),
    ("The shirt is black.", {"entities": [(14, 19, "COLOR")]}),
    ("The clouds are gray.", {"entities": [(16, 20, "COLOR")]}),
    ("The banana is yellow.", {"entities": [(14, 20, "COLOR")]}),
    ("The sun is yellow.", {"entities": [(11, 17, "COLOR")]}),


    ("The grapes are purple.", {"entities": [(15, 21, "COLOR")]}),
    ("The door is white.", {"entities": [(13, 18, "COLOR")]}),
    ("The sky at dawn is pink.", {"entities": [(18, 22, "COLOR")]}),

    ("The sky at dusk is pink.", {"entities": [(18, 22, "COLOR")]}),
    ("The sky at midnight is black.", {"entities": [(22, 27, "COLOR")]}),
    ("The sky at noon is blue.", {"entities": [(18, 22, "COLOR")]}),
    ("The sky is blue and sometimes has clouds.", {"entities": [(11, 15, "COLOR")]}),
    ("The sky is blue and the sun is bright.", {"entities": [(11, 15, "COLOR")]}),
    ("The sky is pink and clear.", {"entities": [(11, 15, "COLOR")]}),
    ("The sky is blue and cloudless.", {"entities": [(11, 15, "COLOR")]}),


]

adjusted_train_data = []
for text, entities in TRAIN_DATA:
    doc = nlp.make_doc(text)
    biluo_tags = offsets_to_biluo_tags(doc, entities.get("entities"))

    corrected_entities = []
    for entity, tag in zip(entities.get("entities"), biluo_tags):
        if tag != 'U':  # Ignore single-character entities that couldn't be aligned
            start, end, label = entity
            adjusted_start = text.find(text[start:end])
            adjusted_end = adjusted_start + len(text[start:end])
            corrected_entities.append((adjusted_start, adjusted_end, label))

    adjusted_train_data.append((text, {"entities": corrected_entities}))
TRAIN_DATA = adjusted_train_data

optimizer = nlp.begin_training()
# Start the training loop
for _ in range(20):  # Number of epochs
    # Shuffle the training data
    random.shuffle(TRAIN_DATA)
    for text, annotations in TRAIN_DATA:
        example = Example.from_dict(nlp.make_doc(text), annotations)
        nlp.update([example], drop=0.2) # Decrease dropout rate to 20%

# Save the model
nlp.to_disk("custom_ner_model")


# Test the ner tagger

In [10]:

# Load the custom NER model
nlp = spacy.load("custom_ner_model")

# Test sentences
test_sentences = [
    "The sky is blue",
    "The sunflower is yellow",
    "His favorite color is purple",
    "The leaves turned golden",
    "The flag is red, white, and blue",
    "The walls are green and the flowers are pink",
    "The car is silver and the shirt is black",
    "The clouds are gray and the banana is yellow",
    "The grapes are purple and the door is white",
    "The sky at dawn is pink",

]

# Process each sentence and print recognized entities
for sentence in test_sentences:
    doc = nlp(sentence)
    # print("Entities in '{}':".format(sentence))
    for ent in doc.ents:
        # print(ent.text, ent.label_)
        pass
    # print()

# create a dataframe to store the results sentence and entities
df = pd.DataFrame(columns=['SENTENCE','RECOGNIZED COLORS'])
# Process each sentence and print recognized entities
for sentence in test_sentences:
    doc = nlp(sentence)
    # print("Entities in '{}':".format(sentence))
    entities = []
    for ent in doc.ents:
        entities.append(ent.text)
        # print(ent.text, ent.label_)
    # remove the bracets
    entities = str(entities).replace('[','').replace(']','')
    new_row = {'SENTENCE': sentence, 'RECOGNIZED COLORS': entities}

    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
df

,SENTENCE,RECOGNIZED COLORS
0,The sky is blue,'blue'
1,The sunflower is yellow,'yellow'
2,His favorite color is purple,'purple'
3,The leaves turned golden,'golden'
4,"The flag is red, white, and blue","'white', 'blue'"
5,The walls are green and the flowers are pink,"'green', 'pink'"
6,The car is silver and the shirt is black,'silver'
7,The clouds are gray and the banana is yellow,"'gray', 'yellow'"
8,The grapes are purple and the door is white,"'purple', 'white'"
9,The sky at dawn is pink,'pink'


# LRIA & USTHB

In [11]:
# corpus =  open("USTHB.txt").read()
# corpus= nlp(corpus)
# Initialize spacy with a blank English model
nlp = spacy.blank("en")

# Define a custom pipeline component for entity recognition
@nlp.component("custom_entity_recognition")
def custom_entity_recognition(doc):
    # Define entities and labels
    entities = [("USTHB", "ORG"), ("LRIA", "ORG")]

    # Iterate over entities and add them to the doc
    for entity, label in entities:
        start = doc.text.find(entity)
        if start != -1: # found
            end = start + len(entity) # get end character index of entity
            span = doc.char_span(start, end, label=label) # create Span for entity
            if span is not None: # entity is not overlapping
                doc.ents = list(doc.ents) + [span] # add span to doc.ents
    return doc

# Add the custom component to the pipeline
nlp.add_pipe("custom_entity_recognition") # add last to the pipeline

# Assuming you have a custom corpus for USTHB and LRIA as ORG entities
custom_corpus = [
    ("USTHB is a university in Algeria", {}),
    ("LRIA is a research institute", {}),
    ("USTHB and LRIA are located in Algiers", {}),
    ("USTHB and LRIA are in the same campus", {}),
    ("USTHB and LRIA are in the same city", {}),
    ("USTHB and LRIA are in the same country", {}),
    ("USTHB and LRIA are in the same continent", {}),
    ("USTHB and LRIA are in the same planet", {}),
    ("USTHB and LRIA are in the same solar system", {}),
    ("USTHB and LRIA are in the same galaxy", {}),
    ("USTHB and LRIA are in the same universe", {}),
    ("USTHB and LRIA are in the same multiverse", {}),
    ("USTHB and LRIA are in the same megaverse", {}),
    ("USTHB and LRIA are in the same omniverse", {}),
]

optimizer = nlp.begin_training()
# Prepare training data
for text, annotations in custom_corpus: # iterate over the data
    doc = nlp.make_doc(text) # create a Doc object from the text
    example = Example.from_dict(doc, annotations) # create Example object from the annotations
    nlp.update([example], drop=0.5 , sgd=optimizer) # update the model

# Test the trained model
doc = nlp("USTHB and LRIA are in the same city")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

for ent in doc.ents:
    print(ent.text, ent.label_)


Entities [('USTHB', 'ORG'), ('LRIA', 'ORG')]
USTHB ORG
LRIA ORG
